## MPC Tensor - Party 1 - Data Scientist

#### 0. Import Libraries

In [1]:
import torch 
import syft as sy 


#### 1.1 Launch DS Duet server

In [2]:
#1.1 - Launch a Duet Server
duet_p1 = sy.launch_duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



#### 1.2 Join to DO Duet server

In [3]:
duet_p2 = sy.join_duet(loopback=True)

♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [4]:
duet_p1

<DomainClient: <UID: 66c38d9f913e4a778fbb1c6da017cf27>>

## 2 SMPC

### 2.1 Create Session

In [5]:
from sympc.session import Session
from sympc.session import SessionManager

from sympc.tensor import MPCTensor


In [6]:
session = Session(parties=[duet_p1, duet_p2])
print(f"Session: {session} \n"
      f"Session parties: {session.parties} \n"
      f"Session protocol: {session.protocol} \n"
      f"ring_size: {session.ring_size} \n")

Session: <sympc.session.session.Session object at 0x000001497D69A040> 
Session parties: [<DomainClient: <UID: 66c38d9f913e4a778fbb1c6da017cf27>>, <Duet: <UID: cb4f5d38552c455c869a451833911184>>] 
Session protocol: <sympc.protocol.fss.fss.FSS object at 0x000001497D9CF370> 
ring_size: 18446744073709551616 



### 2.2 Send session to all parties

In [7]:
SessionManager.setup_mpc(session)

#### Sum, Substract and Multiply operations

In [9]:
x_secret = duet_p2.store["#test_tensor_1"]

In [15]:
x_secret

AttributeError: 'TensorPointer' object has no attribute 'public_shape'

In [8]:
y = torch.Tensor([-5, 0, 1, 2, 3])  # some local data

x_secret = duet_p2.store["#test_tensor_1"]  # secret data to test sum, substract and multiply
x = MPCTensor(secret=x_secret, shape=(1,), session=session)  # MPC Tensor from x_secret
# x.share_ptrs
x

[2022-03-13T16:26:30.446153+0100][CRITICAL][logger]][2568] 'No such item found for id:#test_tensor_1'


KeyError: 'No such item found for id:#test_tensor_1'

In [9]:
print(f"X: {x}\n"
      f"Y: {y}")

X: [MPCTensor]
Shape: (1,)
Requires Grad: False
	| <DomainClient: <UID: 19f650d87e9f45f092893251c6c6670d>> -> ReplicatedSharedTensorShareTensorUnionPointer
	| <Duet: <UID: cae870479896428a9443094c7ef4b36f>> -> AnyPointer
Y: tensor([-5.,  0.,  1.,  2.,  3.])


In [10]:
#Private + Public operation
print(f"X + Y = {(x+y).reconstruct()}\n"
      f"X - Y = {(x-y).reconstruct()}\n"
      f"X * Y = {(x*y).reconstruct()}")

[2022-03-13T15:46:00.175310+0100][CRITICAL][logger]][8604] UnknownPrivateException has been triggered.


UnknownPrivateException: UnknownPrivateException has been triggered.

#### Matrix multiplication

In [ ]:
x_secret = duet_p2.store["#test_tensor_2"]  # secret data with no access
x_mul = MPCTensor(secret=x_secret, shape=(2,2), session=session)  # MPC Tensor build from x_secret
y_mul = torch.tensor([[5,6],[7,8]])
#y_mul = MPCTensor(secret=x_secret, shape=(2,2), session=session)

In [14]:
print(f"X: {x_mul}\n"
      f"Y: {y_mul}")

X: [MPCTensor]
Shape: (2, 2)
Requires Grad: False
	| <DomainClient: <UID: fa496f8af5dd458fbfd8ad859309f662>> -> ShareTensorPointer
	| <Duet: <UID: e30e0c03f30841a1a40ab7f708235722>> -> ShareTensorPointer
Y: tensor([[5, 6],
        [7, 8]])


In [18]:
print(f"X @ X = \n {(x_mul @ y_mul).reconstruct()}")

X @ X = 
 tensor([[19., 22.],
        [43., 50.]])
